1. Topological sorting
2. Dijkstra算法
3. graph与union find,是否能分组


## Topological sorting
#### 先建立入度、出度字典，然后不断找入度/出度为0的顶点，并删除其对应的边
210. Course Schedule II，不断找入度为0的顶点
1136. Parallel Courses，最少需要多少个学期修完课
269. Alien Dictionary，对一系列不等关系建立新的排序
444. Sequence Reconstruction，给定seqs各边，问是否能构成唯一的org顺序
310. Minimum Height Trees，找到能够让树高度最小的节点，实质上是不断删除叶子，留下最后的节点，依然是建立邻接矩阵，不断删除
802. Find Eventual Safe States,给定一堆有向边，找出最终会走到终点的点，先找出出度为0的点，对于这些点，删去其起点，再更新出度为0的点

In [2]:
# 210. Course Schedule II
# 起点每移除一条边，对应端点也移除一条边，当端点的入度为0时，将其变为起点
import collections
def findOrder(numCourses, prerequisites):
    """
    :type numCourses: int
    :type prerequisites: List[List[int]]
    :rtype: List[int]
    """
    first = {i:set() for i in range(numCourses)}
    second = {i:set() for i in range(numCourses)}
    for i,j in prerequisites:# in order to take i, need to take j
        first[j].add(i)
        second[i].add(j)

    queue = collections.deque([i for i in range(numCourses) if len(second[i])==0])

    res = []
    while queue:
        node = queue.popleft()
        res.append(node)
        for course in first[node]:
            second[course].remove(node)
            if len(second[course])==0:
                queue.append(course)
    return res if len(res)==numCourses else []
findOrder(6,[[3,1],[3,0],[2,3],[4,3],[2,4],[1,5],[0,5]])

[5, 0, 1, 3, 4, 2]

In [3]:
# 1136. Parallel Courses，最少需要多少个学期修完课
def minimumSemesters(N, relations):
    """
    :type N: int
    :type relations: List[List[int]]
    :rtype: int
    """
    parent = {i:set() for i in range(1,N+1)}#每门课先修,{1:3,2:3}
    child = {i:set() for i in range(1,N+1)}#每门课需要修的,{3:{1,2}}
    for u,v in relations:
        child[v].add(u)
        parent[u].add(v)

    step = 0
    while len(child)!=0:
        cur = [i for i in child if len(child[i])==0]#这学期可以修的
        if not cur:
            return -1
        for i in cur:
            child.pop(i,None)
            for j in parent[i]:
                child[j].remove(i)
            parent.pop(i,None)
        step+=1
    return step
minimumSemesters(N = 3, relations = [[1,3],[2,3]])

# follow up: If only pick at most k courses in one semester, the shortest time to finish all courses
# Let in_degree[i] represent the in-degree of node i. Let depth[i] represent the longest path starting at node i.
# The algorithm is:这个应该不对

# Put all nodes in a priority_queue with greater and less.
# Each semester, take K courses from the top of priority queue.
# Update the neighbors' in-degree in priority queue.
# Repeat 2, 3.


2

In [36]:
# Parallel Courses II
import heapq
def minNumberOfSemesters(n, dependencies, k):
    """
    :type n: int
    :type dependencies: List[List[int]]
    :type k: int
    :rtype: int
    """
    parent = {i:set() for i in range(1,n+1)}#u:v,v需要先修的课
    child = {i:set() for i in range(1,n+1)}#每门课之后可以对应的课程
    for u,v in dependencies:
        parent[v].add(u)
        child[u].add(v)

    depths = dict()
    nodes = [i for i in child if len(child[i])==0]
    step = 0
    while nodes:
        for node in nodes:
            child.pop(node)
            depths[node]=step
            for newnode in parent[node]:
                child[newnode].remove(node)
        nodes = [i for i in child if len(child[i])==0]
        step+=1

    child = {i:set() for i in range(1,n+1)}#每门课需要修的,{3:{1,2}}
    for u,v in dependencies:
        child[u].add(v)

    nodes = [i for i in parent if len(parent[i])==0]
    visited = [False for _ in range(n)]
    res = 0
    while nodes:
        if len(nodes)<=k:
            level = nodes
        else:
            heap = []
            level = []
            for node in nodes:
                heapq.heappush(heap,(-depths[node],node))
            for i in range(k):
                level.append(heapq.heappop(heap)[1])
        for node in level:
            parent.pop(node)
            for newnode in child[node]:
                parent[newnode].remove(node)
        res+=1
        nodes = [i for i in parent if len(parent[i])==0]
    return res
minNumberOfSemesters(n = 5, dependencies = [[2,1],[3,1],[4,1],[1,5]], k = 2)

4

In [9]:
# 1462. Course Schedule IV
def checkIfPrerequisite(n, prerequisites, queries):
    """
    :type n: int
    :type prerequisites: List[List[int]]
    :type queries: List[List[int]]
    :rtype: List[bool]
    """
    connected = [[False] * n for i in range(n)]

    for i, j in prerequisites:
        connected[i][j] = True

    for k in range(n):
        for i in range(n):
            for j in range(n):
                connected[i][j] = connected[i][j] or (connected[i][k] and connected[k][j])

    return [connected[i][j] for i, j in queries]
checkIfPrerequisite(3,[[1,2],[0,1]],[[0,2]])

In [58]:
# 269. Alien Dictionary，对一系列不等关系建立新的排序
# 和course schedule一样，建立两个字典，然后不断的找没有上家的点
import collections
def alienOrder(words):
    """
    :type words: List[str]
    :rtype: str
    """
    def compare(word1,word2):#找出所有的不等关系
        start = 0
        while start<min(len(word1),len(word2)):
            if word1[start]!=word2[start]:
                return (word1[start],word2[start])
            start+=1
        return [len(word1)<=len(word2)]

    relations = set()
    vocs = set() # 建立词汇表，要包括所有的字母，不能只比较重要的两个char
    for word in words:
        for i in word:
            vocs.add(i)
            
    for i in range(len(words)-1):#建立所有的不等关系
        temp = compare(words[i],words[i+1])
        if len(temp)==1:
            if temp[0] is False:
                return ""
        else:
            relations.add(temp)

    parents = {i:set() for i in vocs} # i is the parent
    children = {i:set() for i in vocs} # i is the children

    for p,c in relations:#建立两字典
        parents[p].add(c)
        children[c].add(p)
    queue = collections.deque([])
    res = []
    for i in children:
        if len(children[i])==0:
            queue.append(i)
    while queue:#排序
        node = queue.popleft()
        res.append(node)
        for i in parents[node]:
            children[i].remove(node)
            if len(children[i])==0:
                queue.append(i)
    return "".join(res) if len(res)==len(vocs) else ""
alienOrder(["z","z"])

'z'

In [35]:
# 444. Sequence Reconstruction，给定seqs各边，问是否能构成唯一的org顺序
import collections
def sequenceReconstruction(org, seqs):
    """
    :type org: List[int]
    :type seqs: List[List[int]]
    :rtype: bool
    """
    nodes = set()
    for i in seqs:
        nodes.update(set(i))
    parents = {i:set() for i in nodes}
    children = {i:set() for i in nodes}
    for e in seqs:
        for i in range(len(e)-1):
            parents[e[i]].add(e[i+1])
            children[e[i+1]].add(e[i])
    queue = collections.deque()
    for key in children:
        if len(children[key])==0:
            queue.append(key)
    if len(queue)!=1:
        return False
    res = []
    while queue:
        node = queue.popleft()
        res.append(node)
        for i in parents[node]:
            children[i].remove(node)
            if len(children[i])==0:
                queue.append(i)
        if len(queue)>1:
            return False
    return res == org and len(res)==len(nodes)#还需要有and
sequenceReconstruction([1],[[1],[2,3],[3,2]])

False

In [16]:
# 310. Minimum Height Trees，找到能够让树高度最小的节点，实质上是不断删除叶子，留下最后的节点，依然是建立邻接矩阵，不断删除
def findMinHeightTrees(n, edges):
    """
    :type n: int
    :type edges: List[List[int]]
    :rtype: List[int]
    """
    if n<=2:
        return list(range(n))
    graph = dict() # build the graph
    for u,v in edges:
        if u not in graph:
            graph[u] = set()
        if v not in graph:
            graph[v] = set()
        graph[u].add(v)
        graph[v].add(u)
    if len(graph)!=n:
        return []
    nodes = dict() # the degree for each node
    queue = []# leaves
    for i in range(n):
        nodes[i] = len(graph[i])
        if nodes[i]==1:
            queue.append(i)
    while queue:#keeping find the nodes with degree=1
        newqueue = []
        for node in queue:
            for v in graph[node]:
                graph[v].remove(node)
                nodes[v]-=1
                if nodes[v]==1:
                    newqueue.append(v)
            graph.pop(node)
        if len(graph)>2:
            queue = newqueue
        else:
            queue = []
    return list(graph.keys())
findMinHeightTrees(n = 6, edges = [[0, 3], [1, 3], [2, 3], [4, 3], [5, 4]])

[3, 4]

In [9]:
# 802. Find Eventual Safe States,给定一堆有向边，找出最终会走到终点的点
# 先找出出度为0的点，对于这些点，删去其起点，再更新出度为0的点,从这个角度说，很像拓扑排序
def eventualSafeNodes(graph):
    """
    :type graph: List[List[int]]
    :rtype: List[int]
    """
    n = len(graph)
    revgraph = {i:set() for i in range(n)}
    graph = {i:set(graph[i]) for i in range(n)}
    for i in range(len(graph)):
        for j in graph[i]:
            revgraph[j].add(i)

    safe = {i for i in range(n) if len(graph[i])==0}

    queue = [i for i in range(n) if len(graph[i])==0]

    while queue:
        newqueue = []
        for i in queue:#nodes which are safe
            for j in revgraph[i]:
                graph[j].remove(i)
                if len(graph[j])==0:
                    newqueue.append(j)
                    safe.add(j)
        queue = newqueue
    return sorted(list(safe))

import collections
def eventualSafeNodes(graph):
    n = len(graph)
    out_degree = [0] * n
    in_nodes = collections.defaultdict(list) 
    queue = []
    for i in range(n):
        out_degree[i] = len(graph[i])
        if out_degree[i] == 0: queue.append(i)
        for j in graph[i]: in_nodes[j].append(i)  
    for term_node in queue:
        for in_node in in_nodes[term_node]:
            out_degree[in_node] -= 1
            if out_degree[in_node] == 0: queue.append(in_node)
    return sorted(queue)
eventualSafeNodes(graph = [[1,2],[2,3],[5],[0],[5],[],[]])

[2, 4, 5, 6]

### Adjacency Matrix:## Dijkstra算法: 更新value值
743. Network Delay Time，图中一个点到其它各顶点的最短距离


每个点到另外一个点与对应的value
``` python
graph = collections.defaultdict(list) 
for u, v, w in times:
    graph[u].append((v, w))
```
743. Network Delay Time，图中一个点到其它各顶点的最短距离
787. Cheapest Flights Within K Stops，图中从一个点到另一个点经过最多K次中转的最小值
1334. Find the City With the Smallest Number of Neighbors at a Threshold Distance, 给定图，求每一个i到所有j的最短路径长度

In [2]:
# 743. Network Delay Time，图中一个点K到其它各顶点的最短距离的最大值
# Given times, a list of travel times as directed edges times[i] = (u, v, w), where u is the source node, v is the target node, and w is the time it takes for a signal to travel from source to target.
# Now, we send a signal from a certain node K. How long will it take for all nodes to receive the signal? If it is impossible, return -1.
# Dijkstra算法,各边用邻接矩阵存储，可以得到从某个点出发到图中各点的最短距离和每个节点的父节点
import collections
def networkDelayTime(times, N, K):
    nodes = [i for i in range(1,N+1)]
#     nodes = {i[0] for i in G} | {i[1] for i in G}
    adj = {i: dict() for i in nodes}
    for i in times:
        adj[i[0]][i[1]]=i[2]
    dis = {i: adj[K].setdefault(i,float('inf')) for i in nodes if i!=K}
    visited = []
    parents_node = {}
    for i in range(len(dis)):
        sorted_dis = sorted(dis.items(),key=lambda item:item[1])
        for p,d in sorted_dis:
            if p not in visited:
                if d==float('inf'):
                    return -1
                min_dis_point = p
                min_dis = d
                visited.append(p)
                break
        
        for n in adj[min_dis_point].keys():
            if n!=K:
                temp = min_dis + adj[min_dis_point][n]
                if dis[n]>temp:
                    dis[n]=temp
                    parents_node[n] = min_dis_point

    for i in nodes:
        if i not in parents_node.keys() and i!=K:
            parents_node[i]=K
    return max(dis.values())

def networkDelayTime(times, N, K):
    t, graph, q = [0] + [float("inf")] * N, collections.defaultdict(list), collections.deque([(0, K)])
    # t用来存储最小值，q用来存储到node的距离
    for u, v, w in times:
        graph[u].append((v, w))
    print(graph)
    while q:
        print(q)
        time, node = q.popleft()
        if time < t[node]:#如果到达某个点的距离比当前存储值小，说明要更新，更新的时候会把之前走过的再走一次
            t[node] = time
            for v, w in graph[node]:
                q.append((time + w, v))
    mx = max(t)
    return mx if mx < float("inf") else -1

import heapq
def networkDelayTime(times, N, K):
    q, t, adj = [(0, K)], {}, collections.defaultdict(list)
    for u, v, w in times:
        adj[u].append((v, w))
    while q:
        time, node = heapq.heappop(q)
        if node not in t:
            t[node] = time#第一个出来的一定是最小值，因为走弯路要的更长
            for v, w in adj[node]:
                heapq.heappush(q, (time + w, v))
    return max(t.values()) if len(t) == N else -1
networkDelayTime(times = [[2,1,5],[2,3,1],[3,4,1],[4,1,1],[1,5,2]], N = 5, K = 2)

5

In [3]:
# 1514. Path with Maximum Probability
def maxProbability(n, edges, succProb, start, end):#从start到end的路径最大概率,和前面K点到别的点的距离一样
    """
    :type n: int
    :type edges: List[List[int]]
    :type succProb: List[float]
    :type start: int
    :type end: int
    :rtype: float
    """
    q, t, adj = [(-1, start)], {}, collections.defaultdict(list)
    for index,(i, j) in enumerate(edges):
        adj[i].append((j, succProb[index]))
        adj[j].append((i, succProb[index]))
    while q:
        time, node = heapq.heappop(q)#一定是概率最大的先弹出来，所以不会出现覆盖的情况，一旦出现了，肯定是最大的概率
        if node not in t:
            t[node] = abs(time)
            for v, w in adj[node]:
                heapq.heappush(q, (-abs(time) * w, v))
    return t[end] if end in t else 0

maxProbability(n = 3, edges = [[0,1],[1,2],[0,2]], succProb = [0.5,0.5,0.2], start = 0, end = 2)

0.25

In [1]:
# 1135. Connecting Cities With Minimum Cost，连通里最小生成树的总长度

# Prim 算法
def minimumCost(N, connections):
    """
    :type N: int
    :type connections: List[List[int]]
    :rtype: int
    Prim's Algorithm:
    1) Initialize a tree with a single vertex, chosen
    arbitrarily from the graph.
    2) Grow the tree by one edge: of the edges that
    connect the tree to vertices not yet in the tree,
    find the minimum-weight edge, and transfer it to the tree.
    3) Repeat step 2 (until all vertices are in the tree).
    """
    graph = collections.defaultdict(list)
    for u,v,w in connections:
        graph[u].append((v,w))
        graph[v].append((u,w))

    minimum = {i:float('inf') for i in range(1,N+1)}#从i节点出发到各节点的距离
    visited = dict()
    heap = [(0,1)]
    heapq.heapify(heap)
    total = 0
    while heap and len(visited)<N:
        val,node = heapq.heappop(heap)
        if node not in visited:#未访问节点里最小的
            visited[node] = True
            total+=val #Grow tree by one edge.
            minimum[node] = total
            for v,w in graph[node]:#添加边
                if v not in visited:
                    heapq.heappush(heap,(w,v))
    return total if len(visited)==N else -1

def minimumCost(N, connections):
    '''
    Kruskal's Algorithm:本质上是一个Union find的过程
    1) Create a forest F (a set of trees), where each vertex in 
    the graph is a separate tree.
    2) Create a set S containing all the edges in the graph.
    3) While S is nonempty and F is not yet spanning (fully connected):
        3A) Remove an edge with minimum weight from S
        3B) If the removed edge connects two different trees then 
        add it to the forest F, combining two trees into a single tree.
    '''
    def find(city):
        # Recursively re-set city's parent to its parent's parent.
        # Build the bush: ideally each tree/set is of height 1.
        if parent[city] != city:
            parent[city] = find(parent[city])
        return parent[city]

    def union(c1, c2):
        root1, root2 = find(c1), find(c2)
        if root1 == root2:
            return False
        parent[root2] = root1  # Always join roots!
        return True

    # [1] Keep track of disjoint sets. Initially each city is its own set.
    parent = {city: city for city in range(1, N+1)}
    # [2] Sort connections so we are always picking minimum cost edge.
    connections.sort(key=lambda x: x[2])#按照权重排序
    total = 0
    for city1, city2, cost in connections:  # [3A]  
        if union(city1, city2):  # [3B]如果不连通，则添加
            total += cost
    # Check that all cities are connected.
    root = find(N)
    return total if all(root == find(city) for city in range(1, N+1)) else -1
minimumCost(N = 3, connections = [[1,2,5],[1,3,6],[2,3,1]])

6

In [8]:
# 1168. Optimize Water Distribution in a Village，可以选择建设管道，或者自己独立建设，问最小生成树的长度
# 将自己建设假设为与0号well连通，求最小生成树
def minCostToSupplyWater(n, wells, pipes):
    """
    :type n: int
    :type wells: List[int]
    :type pipes: List[List[int]]
    :rtype: int
    """
    parent = {i:i for i in range(n+1)}

    def find(node):
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(i,j):
        parent[find(j)] = find(i)

    total = 0
    pipes = sorted([[0,i+1,wells[i]] for i in range(n)]+pipes,key=lambda x:x[2])
    for i,j,w in pipes:
        if find(i)!=find(j):
            total += w
            union(i,j)
            n-=1
        if n==0:
            return total
minCostToSupplyWater(n = 3, wells = [1,2,2], pipes = [[1,2,1],[2,3,1]])

3

In [31]:
# 787. Cheapest Flights Within K Stops，图中从一个点到另一个点经过最多K次中转的最小值
# 从起点出发，一次次往前走BFS，这里面有很多容易错的地方， 起点level要设置src到src为0
# 最容易错的是level里不能只有当前需要扫描的node，还要有此时的val，否则会出现串位，即一条路径的node使用了另一条路径的最小值
def findCheapestPrice(n, flights, src, dst, K):
    """
    :type n: int
    :type flights: List[List[int]]
    :type src: int
    :type dst: int
    :type K: int
    :rtype: int
    """
    graph = {i:dict() for i in range(n)}
    for u,v,w in flights:
        graph[u][v] = w
    tickets = {i:float('inf') for i in range(n)}
    tickets[src] = 0
    level = {(src,0)}
    for i in range(K+1):
        newlevel = set()
        for node,val in level:#节点和目前到节点的距离
            for v in graph[node]:
                if val+graph[node][v]<tickets[v]:
                    tickets[v] = val+graph[node][v]
                    newlevel.add((v,val+graph[node][v]))
        level = newlevel
    return tickets[dst] if tickets[dst]<float('inf') else -1

import collections
import heapq
def findCheapestPrice(n, flights, src, dst, K):
    f = collections.defaultdict(dict)
    for a, b, p in flights:
        f[a][b] = p
    heap = [(0, src, K + 1)]
    while heap:
        p, i, k = heapq.heappop(heap)
        if i == dst:
            return p
        if k > 0:#每次更新值，输入的是(value,node,step)然后输出最小的
            for j in f[i]:
                heapq.heappush(heap, (p + f[i][j], j, k - 1))
    return -1
    
findCheapestPrice(15,[[10,14,43],[1,12,62],[4,2,62],[14,10,49],[9,5,29],[13,7,53],[4,12,90],[14,9,38],[11,2,64],[2,13,92],[11,5,42],[10,1,89],[14,0,32],[9,4,81],[3,6,97],[7,13,35],[11,9,63],[5,7,82],[13,6,57],[4,5,100],[2,9,34],[11,13,1],[14,8,1],[12,10,42],[2,4,41],[0,6,55],[5,12,1],[13,3,67],[3,13,36],[3,12,73],[7,5,72],[5,6,100],[7,6,52],[4,7,43],[6,3,67],[3,1,66],[8,12,30],[8,3,42],[9,3,57],[12,6,31],[2,7,10],[14,4,91],[2,3,29],[8,9,29],[2,11,65],[3,8,49],[6,14,22],[4,6,38],[13,0,78],[1,10,97],[8,14,40],[7,9,3],[14,6,4],[4,8,75],[1,6,56]],\
                  1,4,10)

169

In [3]:
# 1334. Find the City With the Smallest Number of Neighbors at a Threshold Distance, 给定图，求每一个i到所有j的最短路径长度
# 可以求出任何两个点之间的最短距离
# Floyd-Warshall解法
def findTheCity(n, edges, distanceThreshold):
    dis = [[float('inf')] * n for _ in range(n)]
    for i, j, w in edges:
        dis[i][j] = dis[j][i] = w
    for i in range(n):
        dis[i][i] = 0
    for k in range(n):#对于每一个中间节点，看是否导致了另外两个节点的距离因为这个中间节点变得更小
        for i in range(n):
            for j in range(n):
                dis[i][j] = min(dis[i][j], dis[i][k] + dis[k][j])
    res = {sum(d <= distanceThreshold for d in dis[i]): i for i in range(n)}
    return res[min(res)]
findTheCity(n = 4, edges = [[0,1,3],[1,2,1],[1,3,4],[2,3,1]], distanceThreshold = 4)

3

In [11]:
# 最小环问题 shortest cycle
# 在最外层循环到点k时（尚未开始第k次循环），最短路数组dis中，dis[i][j]表示的是从i到j且仅经过编号在<k区间中的点的最短路
def shortestCycle(n,edges):
    dis = [[float("inf") for _ in range(n)] for _ in range(n)]#权重
    val = [[float("inf") for _ in range(n)] for _ in range(n)]#原来的邻接图Adjacency list
    for i,j,w in edges:
        dis[i][j] = dis[j][i] = w
        val[i][j] = val[j][i] = w
    for i in range(n):#有没有不影响
        dis[i][i] =val[i][i]= 0
    res = float("inf")
    for k in range(n):#当外层循环进行到k的时候，与节点k相连的临接点为i,j,该环的长度为dis[i][j] + val[i][k] + val[k][j]
        for i in range(k):#环为i-k,k-j,i-j不经过k，因为这个时候k还没有并入到更新，所以i-k-j的路是不通的
            for j in range(i):#要注意index
                res = min(res, dis[i][j] + val[i][k] + val[k][j])#更新答案
        for i in range(n):
            for j in range(n):
                dis[i][j] = min(dis[i][j], dis[i][k] + dis[k][j])
    return res
    
shortestCycle(n = 6, edges = [[0,1,1],[1,2,2],[0,2,4],[1,3,7],[1,4,5],[2,4,1],[3,4,3],[3,5,2],[4,5,6]])

7

In [5]:
# 1042. Flower Planting With No Adjacent，给定edge，相邻节点不能同样数字，给出任何方案
import collections
def gardenNoAdj(N, paths):
    """
    :type N: int
    :type paths: List[List[int]]
    :rtype: List[int]
    """
    graph = collections.defaultdict(list)
    for i,j in paths:
        graph[i].append(j)
        graph[j].append(i)
    forbiden = collections.defaultdict(set)
    res = []
    for i in range(1,N+1):
        for k in range(1,5):
            if k not in forbiden[i]:
                res.append(k)
                break
        for t in graph[i]:
            forbiden[t].add(k)
    return res
gardenNoAdj(N = 4, paths = [[1,2],[2,3],[3,4],[4,1],[1,3],[2,4]])

[1, 2, 3, 4]

In [7]:
# 1129. Shortest Path with Alternating Colors，从0节点到各节点最短的颜色交错的长度
# 从起点开始，设置对下一个节点的需求然后寻找，每次走出下一步，就把当前的edge删除掉
def shortestAlternatingPaths(n, red_edges, blue_edges):
    """
    :type n: int
    :type red_edges: List[List[int]]
    :type blue_edges: List[List[int]]
    :rtype: List[int]
    """
    edges = [collections.defaultdict(set),collections.defaultdict(set)]
    for i,j in red_edges:
        edges[0][i].add(j)
    for i,j in blue_edges:
        edges[1][i].add(j)

    res = [float('inf') for _ in range(n)]
    queue = collections.deque([(0,1,0),(0,0,0)])#0 means asking for red, 1 means asking for blue
    while queue:
        node,color,step = queue.popleft()
        res[node] = min(res[node],step)
        for i in list(edges[color][node]):#asking for color
            queue.append((i,1-color,step+1))
            edges[color][node].remove(i)
    return [res[i] if res[i]!=float('inf') else -1 for i in range(n)]
shortestAlternatingPaths(n = 3, red_edges = [[0,1],[0,2]], blue_edges = [[1,0]])

[0, 1, 1]

In [ ]:
# 133. Clone Graph,克隆一个graph
class Node(object):
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

def cloneGraph(node):
    """
    :type node: Node
    :rtype: Node
    """
    if not node:
        return
    visited = dict()
    level = {(node,None)}

    while level:
        newlevel = set()
        for node,prev in level:#node是旧节点，prev是新节点
            if node.val not in visited:
                newnode = Node(node.val)
                visited[newnode.val] = newnode
                if prev:
                    prev.neighbors.append(newnode)
                    newnode.neighbors.append(prev)
                else:
                    root = newnode
            else:
                prev.neighbors.append(visited[node.val])
                visited[node.val].neighbors.append(prev)

            for i in node.neighbors:
                if i.val not in visited:
                    newlevel.add((i,newnode))
        level = newlevel
    return root

def cloneGraph(node):
    if not node:
        return 
    nodeCopy = Node(node.val, [])
    dic = {node: nodeCopy}
    queue = collections.deque([node])
    while queue:
        node = queue.popleft()
        for neighbor in node.neighbors:#对于旧节点的相邻节点
            if neighbor not in dic: # 如果相邻节点还没有创建，那么就创建并保存，添加相邻节点
                neighborCopy = Node(neighbor.val, [])
                dic[neighbor] = neighborCopy
                dic[node].neighbors.append(neighborCopy)
                queue.append(neighbor)
            else:#如果已经创建了，那么添加其为相邻节点
                dic[node].neighbors.append(dic[neighbor])
    return nodeCopy

In [3]:
def criticalConnections(n,connections):

    graph = [[] for _ in range(n)] ## vertex i ==> [its neighbors]

    currentRank = 0 ## please note this rank is NOT the num (name) of the vertex itself, it is the order of your DFS level

    lowestRank = [i for i in range(n)] ## here lowestRank[i] represents the lowest order of vertex that can reach this vertex i

    visited = [False for _ in range(n)] ## common DFS/BFS method to mark whether this node is seen before

    ## build graph:
    for connection in connections:
        ## this step is straightforward, build graph as you would normally do
        graph[connection[0]].append(connection[1])
        graph[connection[1]].append(connection[0])


    def dfs(res, graph, lowestRank, visited, currentRank, prevVertex, currentVertex):

        visited[currentVertex] = True 
        lowestRank[currentVertex] = currentRank

        for nextVertex in graph[currentVertex]:
            if nextVertex == prevVertex:#如果下一个点就是上一个点，说明这两个节点互通
                continue ## do not include the the incoming path to this vertex since this is the possible ONLY bridge (critical connection) that every vertex needs.

            if not visited[nextVertex]:
                dfs(res, graph, lowestRank, visited, currentRank + 1, currentVertex, nextVertex)
                # We avoid visiting visited nodes here instead of doing it at the beginning of DFS - 
                # the reason is, even that nextVertex may be visited before, we still need to update my lowestRank using the visited vertex's information.

            lowestRank[currentVertex] = min(lowestRank[currentVertex], lowestRank[nextVertex]) 
            # take the min of the current vertex's and next vertex's ranking
            if lowestRank[nextVertex] >= currentRank + 1: #如果邻接点的最低rank比目前低，说明为critical connection
                res.append([currentVertex, nextVertex])
            
    res = []
    prevVertex = -1
    currentVertex = 0 ## we start the DFS from vertex num 0 (its rank is also 0 of course)
    dfs(res, graph, lowestRank, visited, currentRank, prevVertex, currentVertex)
    return res
criticalConnections(n = 4, connections = [[0,1],[1,2],[2,0],[1,3]])

[[1, 3]]

# 树状图

In [4]:
# 1245. Tree Diameter
import collections
def treeDiameter(edges):
    """
    :type edges: List[List[int]]
    :rtype: int
    """
    if len(edges)==0:
        return 0
    graph = collections.defaultdict(set)
    for u,v in edges:
        graph[u].add(v)
        graph[v].add(u)

    maximum = [0]

    def getDiameter(node,maximum):#以该节点开始的的最多节点
        if len(graph[node])==0:#如果为0，只有1个节点
            return 1
        # if len(graph[node])==1:#如果只有一个节点
        #     for i in list(graph[node]):
        #         graph[node].remove(i)
        #         graph[i].remove(node)
        #         x = getDiameter(i)
        #     self.maximum = max(self.maximum,x+1)
        #     return x+1
        curlen = [0]#避免1一个节点
        for i in list(graph[node]):#如果有多个节点,返回从该节点出发的最大值
            graph[node].remove(i)
            graph[i].remove(node)
            curlen.append(getDiameter(i,maximum))
        maximum[0] = max(maximum[0],sum(sorted(curlen,reverse=True)[:2])+1)#更新节点为中转的最大值
        return max(curlen)+1#返回节点值

    getDiameter(0,maximum)
    return maximum[0]-1

def treeDiameter(self, edges):
    """
    :type edges: List[List[int]]
    :rtype: int
    """
    if not edges:
        return 0
    graph = collections.defaultdict(set)
    for i,j in edges:
        graph[i].add(j)
        graph[j].add(i)

    level = [i for i in graph if len(graph[i])==1]
    res = 0
    while len(level)>1:#不断剪去叶子节点
        newlevel = []
        for node in level:
            for newnode in graph[node]:
                graph[newnode].remove(node)
                if len(graph[newnode])==1:
                    newlevel.append(newnode)
        level = newlevel
        res+=2
    return res if len(level)==1 else res-1
treeDiameter(edges = [[0,1],[1,2],[2,3],[1,4],[4,5]])

4

In [1]:
# 1377. Frog Position After T Seconds
def frogPosition(n, edges, t, target):
    """
    :type n: int
    :type edges: List[List[int]]
    :type t: int
    :type target: int
    :rtype: float
    """
    graph = collections.defaultdict(set)
    for i,j in edges:
        graph[i].add(j)
        graph[j].add(i)

    visited = [False for _ in range(n+1)]
    dp = [0 for _ in range(n+1)]
    visited[1] = True
    dp[1] = 1

    level = [1]
    step = 0
    while step<t and len(level)>0:
        newlevel = []
        for node in level:
            if len(graph[node]):
                length = len(graph[node])
                for neigh in list(graph[node]):
                    graph[neigh].remove(node)
                    graph[node].remove(neigh)
                    dp[neigh] = dp[node]*1.0/length
                    if len(graph[neigh]):
                        newlevel.append(neigh)
                dp[node] = 0
        level = newlevel
        step+=1
    return dp[target]

In [22]:
# 332. Reconstruct Itinerary,给定行程表，确定行程，一个的结束是另一个的开始,可能有重复行程

import collections
def findItinerary(tickets):
    """
    :type tickets: List[List[str]]
    :rtype: List[str]
    """
    graph = dict()
    for i,j in tickets:
        if i not in graph:
            graph[i] = collections.Counter()
        graph[i][j]+=1

    for key in graph:
        graph[key] = sorted(graph[key].items(),key=lambda x:x[0])

    def dfs(path,visited):
        if len(path)==len(tickets)+1:#如果到了终点长度，就返回
            return path
        if path[-1] not in graph or len(graph[path[-1]])==0:
            return False
        for key,val in graph[path[-1]]:#对所有的行程进行遍历，第一个满足的就返回
            cur = path[-1]+"-"+ key
            if cur not in visited or visited[cur]<val:
                t = dfs(path+[key],visited+collections.Counter({cur:1}))
                if t:
                    return t

    return dfs(["JFK"],collections.Counter())

def findItinerary(tickets):
    """
    :type tickets: List[List[str]]
    :rtype: List[str]
    """
    graph = dict()
    for i,j in tickets:
        if i not in graph:
            graph[i] = []
        graph[i].append(j)

    def dfs(path,graph):
        if len(path)==len(tickets)+1:#如果到了终点长度，就返回
            return path
        if path[-1] not in graph or len(graph[path[-1]])==0:
            return False
        for key in sorted(graph[path[-1]]):#对所有的行程进行遍历，第一个满足的就返回
            graph[path[-1]].remove(key)#更新graph
            t = dfs(path+[key],graph)
            graph[path[-1]].append(key)
            if t:
                return t

    return dfs(["JFK"],graph)
    
tickets = [["EZE","AXA"],["TIA","ANU"],["ANU","JFK"],["JFK","ANU"],["ANU","EZE"],["TIA","ANU"],["AXA","TIA"],["TIA","JFK"],["ANU","TIA"],["JFK","TIA"]]
findItinerary(tickets)


def findItinerary(tickets):
    targets = collections.defaultdict(list)
    for a, b in sorted(tickets)[::-1]:
        targets[a] += [b]
    route, stack = [], ['JFK']
    while stack:
        while targets[stack[-1]]:#如果还有下一班航程，就将去找下一个
            stack += [targets[stack[-1]].pop()]
        route += [stack.pop()]
    return route[::-1]
findItinerary(tickets)
# findItinerary([["MUC", "LHR"], ["JFK", "MUC"], ["SFO", "SJC"], ["LHR", "SFO"]])

['JFK', 'ANU', 'EZE', 'AXA', 'TIA', 'ANU', 'JFK', 'TIA', 'ANU', 'TIA', 'JFK']

## graph与union find
261. Graph Valid Tree，给定一堆无向edge，判断是否构成一棵无环的树
547. Friend Circles，给定邻接矩阵，问有多少个环，从一个点出发，将所有的点DFS遍历，如果遍历到了，加入该环，并加入visited
323. Number of Connected Components in an Undirected Graph，一堆edges判断有多少个group，建立邻接矩阵，对每一个节点进行DFS遍历
785. Is Graph Bipartite? 给定不能共存的pair，问是否能分成两组，染色，将新的节点染色，然后dfs其邻接点

In [58]:
# 399. Evaluate Division
def calcEquation(equations, values, queries):
    """
    :type equations: List[List[str]]
    :type values: List[float]
    :type queries: List[List[str]]
    :rtype: List[float]
    """
    graph = dict()#建立graph
    def add_edge(u,v,w):
        if u not in graph:
            graph[u] = dict()
        graph[u][v] = w

    for i,edge in enumerate(equations):
        add_edge(edge[0],edge[1],values[i])
        add_edge(edge[1],edge[0],1/values[i])

    res = []
    for x,y in queries:#一个个进行处理
        if x not in graph or y not in graph:
            res.append(-1.0)
        else:
            queue = collections.deque([(x,1.0)])
            visited = set()
            success = False
            while queue:
                key,val = queue.popleft()
                if key == y:
                    res.append(val)
                    success = True
                    break
                visited.add(key)
                for i in graph[key]:
                    if i not in visited:
                        queue.append((i,graph[key][i]*val))
            if not success:
                res.append(-1.0)
    return res
calcEquation([["a","b"],["c","d"]],[1.0,1.0],[["a","c"],["b","d"],["b","a"],["d","c"]])

[-1.0, -1.0, 1.0, 1.0]

In [5]:
# 261. Graph Valid Tree，给定一堆无向edge，判断是否构成一棵无环的树
def validTree(n, edges):
    """
    :type n: int
    :type edges: List[List[int]]
    :rtype: bool
    """
    graph = {i:set() for i in range(n)}
    for edge in edges:
        graph[edge[0]].add(edge[1])
        graph[edge[1]].add(edge[0])
    visited = [False for _ in range(n)]
    queue = {0}
    visited[0] = True
    res = set()
    while queue:
        newqueue = set()
        for i in queue:
            for j in graph[i]:
                if j not in res:
                    if visited[j]:
                        return False
                    newqueue.add(j)
                    visited[j] = True
        res.update(queue)
        queue = newqueue
    return len(res) == n#要防止出现独立的tree

def validTree(n, edges):
    graph = {i:set() for i in range(n)}
    for edge in edges:
        graph[edge[0]].add(edge[1])
        graph[edge[1]].add(edge[0])
    visited = [False for _ in range(n)]
    queue = {0}
    visited[0] = True

    while queue:
        newqueue = set()
        for i in queue:
            for j in list(graph[i]):
                if visited[j]:
                    return False
                visited[j] = True
                newqueue.add(j)
                graph[j].remove(i)
        queue = newqueue

    return sum(visited) == n
        

In [17]:
# 547. Friend Circles，给定邻接矩阵，问有多少个环，从一个点出发，将所有的点DFS遍历，如果遍历到了，加入该环，并加入visited
def findCircleNum(M):
    """
    :type M: List[List[int]]
    :rtype: int
    """
    count = 0
    N = len(M)
    visited = set()
    def dfs(idx,count):
        stack = [idx]
        count+=1
        while stack:
            f = stack.pop()
            for i in range(N):
                if M[f][i] and i not in visited:
                    stack.append(i)
                    visited.add(i)
        return count

    for i in range(N):
        if i not in visited:
            count = dfs(i,count)
    return count

# 从0到N，把数字存入stack，将所在行其余为1的数都存入stack，这些行都属于同一个circle，存入过的会放在visited里

def findCircleNum(M):
    N = len(M)
    seen = set()
    def dfs(node):
        for nei, adj in enumerate(M[node]):
            if adj and nei not in seen:
                seen.add(nei)
                dfs(nei)
    
    ans = 0
    for i in range(N):
        if i not in seen:
            dfs(i)
            ans += 1
    return ans
findCircleNum([[1,1,0],[1,1,0],[0,0,1]])

2

In [1]:
# 323. Number of Connected Components in an Undirected Graph，一堆edges判断有多少个group，建立邻接矩阵，对每一个节点进行DFS遍历
def countComponents(n, edges):
    """
    :type n: int
    :type edges: List[List[int]]
    :rtype: int
    """
    visited = [False for _ in range(n)]
    graph = {i:set() for i in range(n)}
    for i,j in edges:
        graph[i].add(j)
        graph[j].add(i)
    print(graph)

    def dfs(i):
        if visited[i]:
            return
        visited[i] = True
        for j in graph[i]:
            dfs(j)
    res = 0  
    for i in range(n):
        if not visited[i]:
            res+=1
            dfs(i)
    return res
countComponents(4,[[0,1],[2,3],[1,2]])

{0: {1}, 1: {0, 2}, 2: {1, 3}, 3: {2}}


1

In [6]:
# 785. Is Graph Bipartite? 给定不能共存的pair，问是否能分成两组，染色，将新的节点染色，然后dfs其邻接点
def isBipartite(graph):
    """
    :type graph: List[List[int]]
    :rtype: bool
    """
    n = len(graph)
    visited = [False for _ in range(n)]
    boxes = [set(),set()]
 
    def dfs(i,box):
        if visited[i]:
            return True
        visited[i] = True
        boxes[box].add(i)
        temp = True
        for j in graph[i]:
            if j in boxes[box]:
                return False
            temp = temp &dfs(j,1-box)
        return temp
    
    for i in range(n):
        if not visited[i]:
            if not dfs(i,0):
                return False
    return True

def isBipartite(graph):# graph里是每个index对应的不相容的点
    color = {}
    def dfs(pos):
        for i in graph[pos]:#对pos的相邻节点进行处理，如果在里面，就看是否是对的，如果不在里面，赋值处理
            if i in color:
                if color[i] == color[pos]:
                    return False
            else:
                color[i] = 1 - color[pos]
                if not dfs(i):
                    return False
        return True
    for i in range(len(graph)):
        if i not in color:#对没有出现过的，放在0里
            color[i] = 0
            if not dfs(i):#遍历
                return False
    return True
isBipartite([[1,3], [0,2], [1,3], [0,2]])

True

In [17]:
def makeConnected(n, connections):
    """
    :type n: int
    :type connections: List[List[int]]
    :rtype: int
    """
    if len(connections)<n-1:
        return -1

    def find(node):
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]
    
    def union(i,j):
        i = find(i)
        j = find(j)
        if rank[i] < rank[j]:
            parent[i] = j
        elif rank[i] > rank[j]:
            parent[j] = i
        else:
            parent[i] = j
            rank[j]+=1
    
    parent = {i:i for i in range(n)}
    rank = {i:0 for i in range(n)}
    for i,j in connections:
        if parent[i]!=parent[j]:
            union(i,j)
    
    return len({find(x) for x in parent})-1
makeConnected(n = 6, connections = [[0,1],[0,2],[0,3],[1,2],[1,3]])

In [1]:
# 1319. Number of Operations to Make Network Connected,核心是数有多少个圈子
def makeConnected(n, connections):
    """
    :type n: int
    :type connections: List[List[int]]
    :rtype: int
    """
    if len(connections)<(n-1):#如果数量不够
        return -1

    graph = collections.defaultdict(set)
    for i,j in connections:
        graph[i].add(j)
        graph[j].add(i)

    visited = [False for _ in range(n)]

    def dfs(node):
        if visited[node]:
            return
        visited[node]=True
        for i in graph[node]:
            dfs(i)
        return

    count = 0
    for i in range(n):
        if not visited[i]:
            dfs(i)
            count+=1
    return count-1

In [72]:
# 1059. All Paths from Source Lead to Destination,是否所有从source出发的点都会结束在destination
def leadsToDestination(n, edges, source, destination):
    """
    :type n: int
    :type edges: List[List[int]]
    :type source: int
    :type destination: int
    :rtype: bool
    """
    departure = collections.defaultdict(set)
    for i,j in edges:
        departure[i].add(j)
    if len(departure[destination])!=0:
        return False

    queue = collections.deque([[source,{source}]])
    while queue:
        node, visited = queue.popleft()
        if len(departure[node])==0 and destination!=node:
            return False
        for i in departure[node]:
            if i in visited:
                return False
            queue.append([i,visited|{i}])
    return True

# recursive的方法
def leadsToDestination(n, edges, source, destination):
    def dfs(i,seen):
        seen.add(i)
        for j in graph[i]:
            if j == i or j in seen or not dfs(j,seen):#这里的seen可以删除
                return False
        seen.discard(i)#调用的seen为到它本身时seen的值，比如1和2都在循环里，需要等到1跑完recursive才行，所以需要添加i然后再去除
        return len(graph[i]) != 0 or i == destination
    graph, seen = collections.defaultdict(set), set()
    for a, b in edges:
        graph[a].add(b)
    return dfs(source,seen)
leadsToDestination(n = 2, edges = [[0,1],[1,1]], source = 0, destination = 1)

False

In [123]:
# 1466. Reorder Routes to Make All Paths Lead to the City Zero
def minReorder(n, connections):
    """
    :type n: int
    :type connections: List[List[int]]
    :rtype: int
    """
    graph = {i:set() for i in range(n)}
    graph2 = {i:set() for i in range(n)}
    for i,j in connections:
        graph[i].add(j)#从i走到j
        graph2[j].add(i)#从i走到j

    visited = {0}
    curlevel = {0}
    queue = [0]
    res = 0
    while queue:
        node = queue.pop()
        for i in graph2[node]:
            if i not in visited:
                visited.add(i)
                queue.append(i)
                graph[i].remove(node)
                curlevel.add(i)
        if len(queue) == 0 and len(visited)!=n:
            newcurlevel = set()
            for node in curlevel:
                for i in graph[node]:
                    queue.append(i)
                    graph2[i].remove(node)
                    newcurlevel.add(i)
                    visited.add(i)
                    res+=1
            curlevel = newcurlevel
    return res
minReorder(n = 6, connections = [[0,1],[1,3],[2,3],[4,0],[4,5]])

3

In [3]:
# 685. Redundant Connection II，有向的情况下，有向树多加了一条边，要么形成了环，要么出现了入度为2的节点
# 1. 判断是否有环，如果无环，那么使得入度为2的节点的最后那条边就是
# 2. 如果有环，并且没有入度为2的节点，那么使得形成环的边就是
# 3. 如果有环，且存在入度为2的节点，那么入度为2的节点的两条边里在环里的就是
# 判断有向图的环不能够用union find，没有环的标准是：存在节点入度为0，且出度大于1（避免单枝root）
import collections
def findRedundantDirectedConnection(edges):
    """
    :type edges: List[List[int]]
    :rtype: List[int]
    """
    n = len(edges)
    indegree = collections.defaultdict(list)
    outdegree = collections.defaultdict(list)
    for i,j in edges:
        indegree[j].append(i)
        outdegree[i].append(j)
    t = [i for i in range(1,n+1) if len(indegree[i])==2]#入度为2的节点
    circle = [i for i in range(1,n+1) if len(indegree[i])==0 and len(outdegree[i])>1]      
    def find(node):
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    if circle:#如果没有环，那返回入度为2的节点的最后一条边
        return [indegree[t[0]][1],t[0]]

    if not t:#不存在入度为2的节点，返回组成环的最后一条边
        parent = {i:i for i in range(1,n+1)}
        for i,j in edges:
            if find(i)!=find(j):
                parent[j] = i
            else:
                return [i,j]
    # 存在入度为2的节点，返回该节点中组成环的
    parent = {i:i for i in range(1,n+1)}
    cur = []
    for i,j in edges:
        if j==t[0]:
            cur.append(i)
        else:
            if find(i)!=find(j):
                parent[j] = i
    for i in cur:
        if find(i)==find(t[0]):
            return [i,t[0]]

findRedundantDirectedConnection([[1,2], [2,3], [3,4], [4,1], [1,5]])

[4, 1]

In [10]:
# 无向图上两个点对称轴上的点的总和
import collections
def getSum(graph,n,start,end):
    degree = collections.defaultdict(set)
    for i,j in graph:
        degree[i].add(j)
        degree[j].add(i)
    
    def getDistance(node):
        distances = dict()
        visited = {node}
        level = [node]
        step = 0
        while level:
            newlevel = []
            for i in level:
                distances[i] = step
                for newnode in degree[i]:
                    if newnode not in visited:
                        newlevel.append(newnode)
                        visited.add(newnode)
            level = newlevel
            step+=1
        return distances
    
    disstart = getDistance(start)
    disend = getDistance(end)
    
    res = []
    for key in disstart:
        if disstart[key] == disend[key]:
            res.append(key)
    return sum(res)
getSum([[0,1],[1,2],[0,3],[3,2]],4,0,2)

4